## Información no Estructurada
# Práctica 2 &ndash; Evaluación de buscadores
### Autores: Óscar Calvet, Enrique Ernesto de Alvear

# 1.  Evaluación con juicios de relevancia

### Conexión a Google Drive

Para acceder directamente a la hoja de cálculo con los datos introducidos por los estudiantes.

In [ ]:
import numpy as np
from google.colab import auth
import pandas as pd
import time

auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

### Lectura de datos de la hoja de cálculo

In [ ]:
# Función de comodidad para leer datos de pestañas de una hoja de cálculo
def read_sheet(wb, name, cols=False, colnames=False, coltypes=False):
  df = pd.DataFrame(wb.worksheet(name).get_all_values())[1:]
  if cols: df = df[df.columns[cols]]
  if colnames: df.columns = colnames
  if coltypes: df = df.astype(coltypes)
  return df

# Nos conectamos a la hoja de cálculo
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1c6I2FafjKS1HlGXhYaOM2w1oqsgI7eqDOELQ3Nv7y-M')
print('Reading data...')

# Leemos los rankings
print('  Reading tab', 'Rankings', end='')
rankings = read_sheet(wb, 'Rankings', [0,1,2,3,4], ['qid', 'docid', 'pos', 'score', 'system'], {'qid':'int', 'pos':'int', 'score':'int'})#rankings
rankings = rankings[rankings.docid != '']
print('..... ok')

# Leemos los juicios de relevancia
print('  Reading relevance judgment tabs', end='')
qrels = pd.concat([read_sheet(wb, ws.title, [0, 1, 2], ['qid', 'docid', 'rel'], {'qid':'int'})#juicios de relevancia
                   for ws in wb.worksheets() if ws.title.startswith('Acierto q')]).reset_index(drop=True)
print('..... ok')

# Comprobamos duplicados
pd.set_option('display.max_colwidth', 50)
duplicates = qrels[qrels.duplicated(['qid', 'docid'])]
if duplicates.size: print('\nDuplicate relevance judgments\n-----------------------------\n',
                          duplicates.to_string(index=False, max_colwidth=70))

# Comprobamos que coincidan las URLs en los rankings y juicios de relevancia
# (y de paso hacemos un join en ranking_rels para facilitar la implementación de métricas)
qm = qrels.merge(rankings, how='left')
ranking_rels = rankings.merge(qrels, how='left')
missing = ranking_rels[pd.isna(ranking_rels.rel)]
if missing.size: print('\nMissing relevance judgments\n---------------------------\n',
                       missing[['qid', 'docid']].to_string(index=False, max_colwidth=100))
missing = qm[pd.isna(qm.pos)]
if missing.size: print('\nMissing results\n---------------\n',
                       missing[['qid', 'docid']].to_string(index=False, max_colwidth=100))
ranking_rels["rel"]=ranking_rels["rel"].fillna(0)
ranking_rels["rel"]=ranking_rels["rel"].astype(float).astype(int)

Reading data...
  Reading tab Rankings..... ok
  Reading relevance judgment tabs..... ok


In [ ]:
rankings

,qid,docid,pos,score,system
1,0,http://en.wikipedia.org/wiki/Information_retri...,1,10,bing
2,0,http://en.wikipedia.org/wiki/Information_theory,2,9,bing
3,0,http://nlp.stanford.edu/IR-book/pdf/irbookonli...,3,8,bing
4,0,http://www.britannica.com/topic/retrieval,4,7,bing
5,0,http://nlp.stanford.edu/IR-book/information-re...,5,6,bing
...,...,...,...,...,...
356,9,https://jugadoresdeajedrez.com/aperturas/las-3...,6,5,yandex
357,9,https://asisejuega.com/guias/aperturas-de-ajed...,7,4,yandex
358,9,https://ajedrez.pro/aperturas,8,3,yandex
359,9,https://pressplaytv.in/quin-debe/cul-es-la-mej...,9,2,yandex


In [ ]:
ranking_rels['docid'] = ranking_rels['docid'].astype('string')
ranking_rels['system'] = ranking_rels['system'].astype('string')

### Implementación de las métricas

Una vez procesa el conjunto de las queries y las valoraciones procedemos a crear las funciones para evaluar el rendimiento de los distintos navegadores.

In [ ]:
# Código aquí (utilizando ranking_rels, o rankings y qrels, a preferencia del estudiante)
import math
def prec(ranking):
    aux = ranking.copy()
    aux['rel'] = aux['rel'].apply(lambda x : 1 if x > 0 else 0)
    return aux.groupby(["system", "qid"])["rel"].mean()

def prec_mean(ranking):
    return prec(ranking).groupby(["system"]).mean()

#Entiendo que es la f1-score
def harmonic(ranking):
  return (2*prec(ranking)*recall(ranking)/(prec(ranking)+recall(ranking)))

def harmonic_mean(ranking):
  return harmonic(ranking).groupby(['system']).mean()

def DCG(ranking):
  rk = ranking.copy()
  aux = rk["rel"] / np.log2(rk["pos"] +1)
  rk["DCG"] = aux
  return rk.groupby(["system", "qid"])["DCG"].sum()

def IDCG(ranking):
  rk = ranking.copy()
  rk = pd.DataFrame(rk.sort_values(['qid', 'system', 'rel'], ascending=[True,True,False]))
  rk["pos"] = np.tile(np.arange(1,ranking['pos'].nunique()+1), ranking["system"].nunique()*(ranking["qid"].nunique()))
  aux = rk["rel"] / np.log2(rk["pos"] +1)
  rk["IDCG"] = aux
  return rk.groupby(['system', 'qid'])['IDCG'].sum()

def nDCG_(ranking):
  return DCG(ranking) / IDCG(ranking)

def DCG_mean(ranking):
  aux = ranking["rel"] / np.log2(ranking["pos"] +1)
  rk = ranking.copy()
  rk["DCG"] = aux
  return rk.groupby(["system"])["DCG"].sum()

def IDCG_mean(ranking):
  rk = ranking.copy()
  rk = rk.sort_values(['qid', 'system', 'rel'], ascending=[True,True,False])
  rk['IDCG'] = rk['rel']/np.log2(np.tile(np.arange(1,ranking['pos'].nunique()+1),ranking["system"].nunique()*(ranking["qid"].nunique())) +1)
  return rk.groupby(['system'])['IDCG'].sum()

def nDCG_mean(ranking):
  return nDCG_(ranking).groupby(["system"]).mean()

def RBP_(ranking, p = 0.5):
  rk = ranking.copy()
  rk['RBP'] = rk['rel'].apply(lambda x: 1 if x>0 else 0)*np.power(p, rk['pos']-1)
  return rk.groupby(['system', 'qid'])['RBP'].sum()*(1-p)

def RBP_mean(ranking, p = 0.5):
  return RBP_(ranking, p).groupby(["system"]).mean()

def Mean_RR(ranking):
  rk = ranking[ranking['rel']>0]
  val = rk.groupby(["system", "qid"])['pos'].min()
  return 1/val

def Mean_RR_mean(ranking):
  return Mean_RR(ranking).groupby(["system"]).mean()

def Max_RR(ranking):
  rk = ranking.copy()
  rk["RR"] = rk["rel"]/rk["pos"]
  return rk.groupby(["system", "qid"])["RR"].max()

def Max_RR_mean(ranking):
  rk = ranking.copy()
  rk["RR"] = rk["rel"]/rk["pos"]
  return rk.groupby(["system"])["RR"].mean()

def recall(ranking):
  aux = ranking.copy()[ranking["rel"] > 0]
  aux = aux.groupby(["qid"])["docid"].nunique()
  rk = ranking[ranking["rel"] > 0]
  rk = rk.groupby(["system", "qid"])["rel"].size() / aux
  return rk

def recall_mean(ranking):
  aux = ranking.copy()[ranking["rel"] > 0]
  aux = aux.groupby(["qid"])["docid"].nunique()
  rk = ranking[ranking["rel"] > 0]
  rk = rk.groupby(["system", "qid"])["rel"].size() / aux
  rk = rk.groupby(["system"]).mean()
  return rk

print("Precisión por query y buscador:")
print(prec(ranking_rels))

print("\nPrecisión media por buscador:")
print(prec_mean(ranking_rels))

print("\nnDCG por query y buscador:")
print(nDCG_(ranking_rels))

print("\nnDCG medio por buscador:")
print(nDCG_mean(ranking_rels))

print("\nRBP por query y buscador:")
print(RBP_(ranking_rels))

print("\nRBP medio por buscador:")
print(RBP_mean(ranking_rels))

print("\nMeanRR por query y buscador:")
print(Mean_RR(ranking_rels))

print("\nMeanRR medio por buscador:")
print(Mean_RR_mean(ranking_rels))

print("\nMaxRR por query y buscador:")
print(Max_RR(ranking_rels))

print("\nMaxRR medio por buscador:")
print(Max_RR_mean(ranking_rels))

print("\nrecall por query y buscador:")
print(recall(ranking_rels))

print("\nrecall medio por buscador:")
print(recall_mean(ranking_rels))

print("\nmedia harmonica por query y buscador:")
print(harmonic(ranking_rels))

print("\nmedia harmonica media por buscador:")
print(harmonic_mean(ranking_rels))


Precisión por query y buscador:
system      qid
bing        0      0.2
            1      0.6
            2      0.2
            3      0.2
            4      0.2
            5      0.7
            7      0.8
            8      0.3
            9      0.9
duckduckgo  0      0.4
            1      0.7
            2      0.2
            3      0.2
            4      0.2
            5      0.7
            7      0.9
            8      0.3
            9      1.0
google      0      0.1
            1      0.9
            2      0.5
            3      0.7
            4      0.4
            5      0.8
            7      1.0
            8      0.8
            9      0.8
yandex      0      0.3
            1      0.7
            2      0.3
            3      0.2
            4      0.3
            5      0.6
            7      1.0
            8      0.3
            9      0.8
Name: rel, dtype: float64

Precisión media por buscador:
system
bing          0.455556
duckduckgo    0.511111
google        

## Significatividad de los resultados

Como podemos comprobar, en la gran mayoría de casos Google y DuckDuckGo suelen mostrar los mejores resultados. Para corroborar la significatividad estadística de esta comparación vamos a obtener varias muestras de las métricas con bootstrap por lo que nos quedaremos con 25 experimentos que consistirán en evaluar las métricas implementadas con remuestras bootstrap de las distintas queries quedándonos con 7 de ellas.

In [ ]:
import random

lista_queries_id = list(ranking_rels["qid"].unique())


res_google = {"P":[],"nDCG":[], "RR":[] ,"recall":[]}
res_DDGo = {"P":[],"nDCG":[], "RR":[] ,"recall":[]}
for _ in range(25):
  queries = random.choices(lista_queries_id, k = 7)
  rk = ranking_rels[ranking_rels["qid"].isin(queries) & ranking_rels["system"].isin(["google", "duckduckgo"])]
  prec_googleDDGo = prec_mean(rk)
  res_google["P"].append(prec_googleDDGo["google"])
  res_DDGo["P"].append(prec_googleDDGo["duckduckgo"])
  ndcg_googleDDG = nDCG_mean(rk)
  res_google["nDCG"].append(ndcg_googleDDG["google"])
  res_DDGo["nDCG"].append(ndcg_googleDDG["duckduckgo"])
  RR_googleDDG =  Mean_RR_mean(rk)
  res_google["RR"].append(RR_googleDDG["google"])
  res_DDGo["RR"].append(RR_googleDDG["duckduckgo"])
  recall_googleDDG = recall_mean(rk)
  res_google["recall"].append(recall_googleDDG["google"])
  res_DDGo["recall"].append(recall_googleDDG["duckduckgo"])

#Y se hará un test de Wilcoxon para estudiar la significancia estadística
from scipy.stats import wilcoxon

for metric in ["P", "nDCG", "RR", "recall"]:
  stat, p_value = wilcoxon(res_google[metric], res_DDGo[metric])
  print(f"El pvalor para la métrica {metric} es: {p_value}")
#pvalor menor que cualquier nivel de significancia, podemos rechazar que sean iguales

El pvalor para la métrica P es: 5.960464477539063e-08
El pvalor para la métrica nDCG es: 0.00012022256851196289
El pvalor para la métrica RR es: 0.00022404635334095604
El pvalor para la métrica recall es: 1.7881393432617188e-07


/usr/local/lib/python3.10/dist-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Hemos usado el test de Wilcoxon para obtener los p-valores que se muestran en la celda anterior. Como podemos ver, la diferencia de resultados indica que ambos buscadores poseen una distribución claramente distinta en todas las métricas que hemos evaluado con una gran confianza (mayor que 0.99), por lo que concluimos que las diferencias son notables y relevantes.

## Comparación con pyTerrier

Pasamos ahora a evaluar los conjuntos mediante las métricas implementadas en el paquete ir-measures mediante pyTerrier

In [ ]:
# Utlizamos la librería de motor de búsqueda PyTerrier.
!pip install python-terrier
!pip install ir-measures

import pyterrier as pt
if not pt.started(): pt.init()
pt.logging('ERROR')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.2/335.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



Generamos los datasets para los qrels y para run.

In [ ]:
from ir_measures import *

qrel_prof = ranking_rels.copy()
qrel_prof = qrel_prof.drop(['pos', 'score'], axis=1)
qrel_prof = qrel_prof.rename(columns={'qid': 'query_id', 'docid': 'doc_id', 'rel': 'relevance'})
qrel_prof.query_id = qrel_prof.query_id.astype('string')
run_prof = ranking_rels.copy()
run_prof = run_prof.drop(['pos', 'rel'], axis = 1)
run_prof = run_prof.rename(columns={'qid': 'query_id', 'docid': 'doc_id'})
run_prof.query_id = run_prof.query_id.astype('string')

### Precisión

In [ ]:
print('Precisión media por buscador según pyTerrier\n')
for names in ['bing', 'duckduckgo', 'google', 'yandex']:
  system_name = names
  qrel_trial = qrel_prof[qrel_prof['system'] == system_name]
  run_trial =  run_prof[run_prof['system'] == system_name]
  res = (P@10).calc_aggregate(qrel_prof, run_trial)
  print(names + ': ' + str(res))

print("\nPrecisión media por buscador según nuestra implementación:")
print(prec_mean(ranking_rels))

Precisión media por buscador según pyTerrier

bing: 0.4555555555555555
duckduckgo: 0.5111111111111111
google: 0.6666666666666666
yandex: 0.5

Precisión media por buscador según nuestra implementación:
system
bing          0.455556
duckduckgo    0.511111
google        0.666667
yandex        0.500000
Name: rel, dtype: float64


### Recall

In [ ]:
print('Recall medio por buscador según pyTerrier\n')
for names in ['bing', 'duckduckgo', 'google', 'yandex']:
  system_name = names
  qrel_trial = qrel_prof[qrel_prof['system'] == system_name]
  run_trial =  run_prof[run_prof['system'] == system_name]
  res = (R@10).calc_aggregate(qrel_prof, run_trial)
  print(names + ': ' + str(res))

print("\nRecall medio por buscador según nuestra implementación:")
print(recall_mean(ranking_rels))

Recall medio por buscador según pyTerrier

bing: 0.3059892651017522
duckduckgo: 0.34626402607826756
google: 0.5080496721105595
yandex: 0.3536522406233449

Recall medio por buscador según nuestra implementación:
system
bing          0.305989
duckduckgo    0.346264
google        0.508050
yandex        0.353652
dtype: float64


### Media armónica (F1-score)

In [ ]:
print('Media armónica media por buscador según pyTerrier\n')
for names in ['bing', 'duckduckgo', 'google', 'yandex']:
  system_name = names
  qrel_trial = qrel_prof[qrel_prof['system'] == system_name]
  run_trial =  run_prof[run_prof['system'] == system_name]
  res = (SetF).calc_aggregate(qrel_prof, run_trial)
  print(names + ': ' + str(res))

print("\nMedia armónica media por buscador según nuestra implementación:")
print(harmonic_mean(ranking_rels))

Media armónica media por buscador según pyTerrier

bing: 0.34845575188093514
duckduckgo: 0.3930280651260427
google: 0.5473081552708687
yandex: 0.39134856270611396

Media armónica media por buscador según nuestra implementación:
system
bing          0.348456
duckduckgo    0.393028
google        0.547308
yandex        0.391349
dtype: float64


### Mean Reciprocal Rank

In [ ]:
print('MRR medio por buscador según pyTerrier\n')
for names in ['bing', 'duckduckgo', 'google', 'yandex']:
  system_name = names
  qrel_trial = qrel_prof[qrel_prof['system'] == system_name]
  run_trial =  run_prof[run_prof['system'] == system_name]
  res = (MRR@10).calc_aggregate(qrel_prof, run_trial)
  print(names + ': ' + str(res))

print("\nMRR medio por buscador según nuestra implementación:")
print(Mean_RR_mean(ranking_rels))

MRR medio por buscador según pyTerrier

bing: 0.8333333333333334
duckduckgo: 0.8148148148148148
google: 0.9074074074074076
yandex: 0.7777777777777778

MRR medio por buscador según nuestra implementación:
system
bing          0.833333
duckduckgo    0.814815
google        0.907407
yandex        0.777778
Name: pos, dtype: float64


### nDCG

In [ ]:
print('nDCG medio por buscador según pyTerrier\n')
for names in ['bing', 'duckduckgo', 'google', 'yandex']:
  system_name = names
  qrel_trial = qrel_prof[qrel_prof['system'] == system_name]
  run_trial =  run_prof[run_prof['system'] == system_name]
  res = (nDCG@10).calc_aggregate(qrel_trial, run_trial)
  print(names + ': ' + str(res))

print("\nnDCG medio por buscador según nuestra implementación:")
print(nDCG_mean(ranking_rels))

nDCG medio por buscador según pyTerrier

bing: 0.8103146006483675
duckduckgo: 0.7853858175617808
google: 0.8621961146581032
yandex: 0.7930171632126656

nDCG medio por buscador según nuestra implementación:
system
bing          0.810315
duckduckgo    0.785386
google        0.862196
yandex        0.793017
dtype: float64


Rank-Biased Precision

In [ ]:
print('RBP medio por buscador según pyTerrier\n')
for names in ['bing', 'duckduckgo', 'google', 'yandex']:
  system_name = names
  qrel_trial = qrel_prof[qrel_prof['system'] == system_name]
  run_trial =  run_prof[run_prof['system'] == system_name]
  res = (RBP(rel = 1, p = 0.5)).calc_aggregate(qrel_trial, run_trial)
  print(names + ': ' + str(res))

print("\nRBP medio por buscador según nuestra implementación:")
print(RBP_mean(ranking_rels))

RBP medio por buscador según pyTerrier

bing: 0.6529947916666666
duckduckgo: 0.6547309027777778
google: 0.83203125
yandex: 0.5886501736111112

RBP medio por buscador según nuestra implementación:
system
bing          0.652995
duckduckgo    0.654731
google        0.832031
yandex        0.588650
Name: RBP, dtype: float64


Como podemos comprobar, los resultados concuerdan en todas las métricas.

# 2.  Evaluación con métricas de negocio

### Lectura de los logs simulados

Realizar una evaluación A/B test (con cuatro sistemas) basada en clicks utilizando un log de búsqueda simulado que incluye consultas recibidas por los cuatro buscadores por varios usuarios simulados, y acciones de estos usuarios (clicks y compras) en respuesta a los resultados devueltos. Esta simuilación reproduce diferentes patrones de comportamiento por parte de los usuarios según modelos de browsing en cascada con diferentes parámetros de paciencia y satisfacción.
Calcular las siguientes métricas para los buscadores que aparecen en el log: clicks por consulta, tasa de abandono, Max RR, Mean RR, y "unidades vendidas".

Procedemos a cargar los documentos:

In [ ]:
# !pip install -U -q PyDrive2
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Leemos el log de interacción simulada preparado por el profesor a los dataframes de queries y engagement.
drive.CreateFile({'id':'1KwuB8yLDNDYdnW1h-51vImle1iHzv0Bt'}).GetContentFile('query-requests.csv')
# Backup: la siguiente línea, comentada, lee la versión inicialmente proporcionada del log
# drive.CreateFile({'id':'1r1ZRLI9uiAoFjiZXj1dHhRi2VsPXKQ6t'}).GetContentFile('impressions.csv')
query_requests = pd.read_csv('query-requests.csv')
impressions = query_requests

drive.CreateFile({'id':'1ox0FGfOKNCMPSpGcLcPhUE_oBuuTC2uE'}).GetContentFile('engagement-log.csv')
# Backup: la siguiente línea, comentada, lee la versión inicialmente proporcionada del log
# drive.CreateFile({'id':'1RuI7EKyFzhm6cBOoKkfY740PlcSrRqDx'}).GetContentFile('engagement-log.csv')
engagement = pd.read_csv('engagement-log.csv')

### Cálculo de las métricas de negocio

Pasamos ahora a programar las métricas:

In [ ]:
#Vamos a quitar primero las filas de google+bing
impressions_ = impressions[impressions["system"] != "google+bing"].copy()
engagement_ = engagement[engagement["system"] != "google+bing"].copy()

# Código aquí (usando los datos en 'impressions' y 'engagement').

def clicks_consulta(impressions, engagement):
  rk1 = impressions.copy()
  rk2 = engagement[["system", "qid","click"]].copy()
  rk2 = rk2[rk2["click"] > 0]
  rk2 = rk2.groupby(["system"])["click"]
  return (rk2.sum() / rk1.groupby(["system"]).size()).dropna()#Número de clicks / número de consultas totales

def tasa_abandono(impressions, engagement):
  total_req = impressions.groupby(['system'])['requestid'].nunique()
  total_clicked = engagement.groupby(['system'])['requestid'].nunique()
  return 1-total_clicked/total_req

def Mean_RR_2(impressions, engagement): #impressions no se usa pero para que tengan la misma entrada todos
  rk = engagement.copy()
  rk["RR"] = rk["click"]/rk["pos"]
  return rk.groupby(["system"])["RR"].mean()

def Max_RR_2(impressions, engagement):#impressions no se usa pero para que tengan la misma entrada todos
  rk = engagement.copy()
  rk["RR"] = rk["click"]/rk["pos"]
  return rk.groupby(["system"])["RR"].max()

def unidades_vendidas(impressions, engagement):#impressions no se usa pero para que tengan la misma entrada todos
  rk = engagement[engagement["click"] > 0].copy()
  rk = rk[["system", "purchase"]]
  rk = rk.groupby(["system"])["purchase"].sum()
  return  rk

print("Clicks por consulta:")
print(clicks_consulta(impressions_, engagement_))

print("\nTasa de abandono:")
print(tasa_abandono(impressions_, engagement_))

print("\nMax RR:")
print(Max_RR_2(impressions_, engagement_))

print("\nMean RR:")
print(Mean_RR_2(impressions_, engagement_))

print("\nUnidades vendidas:")
print(unidades_vendidas(impressions_, engagement_))

Clicks por consulta:
system
bing          0.656250
duckduckgo    0.510638
google        0.709677
yandex        0.657143
dtype: float64

Tasa de abandono:
system
bing          0.531250
duckduckgo    0.659574
google        0.483871
yandex        0.571429
Name: requestid, dtype: float64

Max RR:
system
bing          0.500000
duckduckgo    0.200000
google        0.166667
yandex        0.200000
Name: RR, dtype: float64

Mean RR:
system
bing          0.133787
duckduckgo    0.118667
google        0.117082
yandex        0.126294
Name: RR, dtype: float64

Unidades vendidas:
system
bing          17
duckduckgo    21
google        17
yandex        21
Name: purchase, dtype: int64


Podemos apreciar que los resultados con variados y no hay un claro navegador mejor que el resto.

# 3.  Opcional: evaluación online intercalada

Realizar una evaluación A/B de Google vs. Bing utilizando los clicks sobre ránkings intercalados "google+bing" del ejercicio 2, sabiendo que se ha utilizado el método balanceado. A criterio del propio estudiante, decidir el número de consultas a procesar en el ejercicio para dimensionar razonablemente el esfuerzo vs. provecho del estudiante.

Para este apartado nos basta con generar una función que otorgue clicks y compras en función del criterio de clicks balanceado. Las compras (en los casos en las que se hubieran realizado) se han otorgado a cada navegador de la misma manera que los clicks.

In [ ]:
#Usamos solamente los datos google+bing

impressions_3 = impressions[impressions['system']=='google+bing']
engagement_3 = engagement[engagement['system']=='google+bing']

#Para cada request calculamos los clicks y las compras asociadas a cada navegador

intercal = pd.DataFrame(columns=['requestid', 'system', 'purchase', 'pos', 'qid', 'click'])

for id in engagement_3['requestid'].unique():
  clicks_actual = engagement_3[engagement_3['requestid'] == id]
  max_pos = np.max(clicks_actual['pos'])
  max_doc = clicks_actual[clicks_actual['pos'] == max_pos]['docid'].values[0]
  max_qid = clicks_actual[clicks_actual['pos'] == max_pos]['qid'].values[0]
  #Encontrar la posición mínima del último doc clickeado entre los dos buscadores
  gpos=13
  bpos=13
  if not ranking_rels[(ranking_rels['system'] == 'google') & (ranking_rels['docid'] == max_doc) & (ranking_rels['qid'] == max_qid)].empty:
    gpos = ranking_rels[(ranking_rels['system'] == 'google') & (ranking_rels['docid'] == max_doc) & (ranking_rels['qid'] == max_qid)]['pos'].values[0]
  if not ranking_rels[(ranking_rels['system'] == 'bing') & (ranking_rels['docid'] == max_doc) & (ranking_rels['qid'] == max_qid)].empty:
    bpos = ranking_rels[(ranking_rels['system'] == 'bing') & (ranking_rels['docid'] == max_doc) & (ranking_rels['qid'] == max_qid)]['pos'].values[0]
  minpos = min(gpos, bpos)
  for doc_id, pur, qid in zip(clicks_actual['docid'], clicks_actual['purchase'], clicks_actual['qid']):
    if not ranking_rels[(ranking_rels['system'] == 'google') & (ranking_rels['docid'] == doc_id) & (ranking_rels['qid'] == qid)].empty:
      gpos = ranking_rels[(ranking_rels['system'] == 'google') & (ranking_rels['docid'] == doc_id) & (ranking_rels['qid'] == qid)]['pos'].values[0]
      if gpos <= minpos:
        intercal.loc[len(intercal.index)] = [id, 'google', pur, gpos, qid, 1]
    if not ranking_rels[(ranking_rels['system'] == 'bing') & (ranking_rels['docid'] == doc_id) & (ranking_rels['qid'] == qid)].empty:
      bpos = ranking_rels[(ranking_rels['system'] == 'bing') & (ranking_rels['docid'] == doc_id) & (ranking_rels['qid'] == qid)]['pos'].values[0]
      if bpos <= minpos:
        intercal.loc[len(intercal.index)] = [id, 'bing', pur, bpos, qid, 1]

Tras esta evaluación obtenemos la siguiente tabla con clicks y compras asignados a cada navegador.

In [ ]:
intercal

,requestid,system,purchase,pos,qid,click
0,13,google,1,3,1,1
1,27,google,1,3,2,1
2,61,google,0,1,7,1
3,61,bing,0,1,7,1
4,68,google,1,1,3,1
5,68,bing,1,1,3,1
6,75,google,0,2,8,1
7,79,google,1,2,5,1
8,94,bing,1,1,8,1
9,101,google,1,2,1,1


Una vez tenemos los clicks asociados a cada valor podemos utilizar las métricas de negocio. Hemos duplicado el conjunto de las queries mixtas y supuesto que cada una es una búsqueda en Google y otra en Bing con los mismos datos.

In [ ]:
print("Clicks por consulta:")
aux1 = impressions_3.copy()
aux2 = impressions_3.copy()
aux1['system'] = 'google'
aux2['system'] = 'bing'
aux1 = pd.concat([aux1,aux2])
cconsulta = pd.merge(aux1, intercal, on = ['requestid', 'system'], how='outer').fillna(0).groupby(['system','requestid'])['click'].sum().groupby(['system']).mean()
print(cconsulta)

print("\nTasa de abandono:")

print(tasa_abandono(aux1, intercal))

print("\nMax RR:")
print(Max_RR_2(impressions_3, intercal))

print("\nMean RR:")
print(Mean_RR_2(impressions_3, intercal))

print("\nUnidades vendidas:")
print(unidades_vendidas(impressions_3, intercal))

Clicks por consulta:
system
bing      0.171429
google    0.371429
Name: click, dtype: float64

Tasa de abandono:
system
bing      0.857143
google    0.685714
Name: requestid, dtype: float64

Max RR:
system
bing      1.0
google    1.0
Name: RR, dtype: float64

Mean RR:
system
bing      0.722222
google    0.679487
Name: RR, dtype: float64

Unidades vendidas:
system
bing      3
google    9
Name: purchase, dtype: int64


Aparentemente Google es notablemente mejor buscador que Bing en todas las métricas implementadas.

# 4.  Opcional: evaluación de los rankers implementados en la práctica 1

Procesar [el conjunto de URLs](https://docs.google.com/spreadsheets/d/1nWr6r1ZkLH29WTyhqr4oz05HgvQv0tmE-IPHdZUex0c/edit#gid=1095978917) de esta práctica 2 utilizando el código de la práctica 1, de forma que se aplique la misma extracción de términos.

Partiendo de ahí, ejecutar los modelos implementados en la práctica 1 (VSM, BM25, QLD), así como los de PyTerrier, sobre las consultas y documentos de la práctica 2, y evaluarlos con los juicios de relevancia de esta práctica y las métricas implementadas en el ejercicio 1.

De esta forma el estudiante podrá comparar qué implementaciones de rankers de la práctica 1 eran más efectivas en esta nueva colección de documentos y consultas.

Primero copiamos el código de los estimadores de la práctica 1

In [ ]:
#Código de la practica anterior
from math import sqrt, log2
import numpy as np
class VSM:
  def __init__(self, freqvector, docfreqs):
    self.freqvector = freqvector
    self.docfreqs = docfreqs

  def search(self, q):
    # Calculamos los cosenos de todos los documentos.
    ranking = [(url, self.dotproduct(url, q) / self.module(url)) if self.module(url) != 0 else (url,0) for url in self.freqvector]
    # Eliminamos los documentos con coseno = 0.
    ranking = [(url, cos) for url, cos in ranking if cos > 0]
    # Ordenamos.
    ranking.sort(key=lambda x: x[1], reverse=True)
    return ranking

  def dotproduct(self, url, q):
    result = 0
    for word in q:
      result += self.tf(word, url) * self.idf(word)
    return result


  def module(self, url):
    result = 0
    for word in self.freqvector[url]:
      result += self.freqvector[url][word]**2
    return sqrt(result)



  def tf(self, word, url):
    if self.freqvector[url][word] > 0:
      return (1+log2(self.freqvector[url][word]))
    else:
      return 0


  def idf(self,word):
    try:
      self.docfreqs[word]
      return (log2(len(self.freqvector) + 0.5) / (self.docfreqs[word])+1)
    except:
      return 0



class BM25:
  def __init__(self, freqvector, docfreqs, b, k):
    self.freqvector = freqvector
    self.docfreqs = docfreqs
    self.b = b
    self.k = k
    self.avg = 0
    for d in self.freqvector:
      self.avg += np.array(list(self.freqvector[d].values())).sum()
    self.avg /= len(self.freqvector)


  def search(self, q):
    ranking = []

    def RSJ(w):
      try:
        return log2((max(20,len(self.freqvector)) - self.docfreqs[w] + 0.5) / (self.docfreqs[w] + 0.5))
      except:
        return 0

    def f(q, d):
      result = 0
      for word in q:
        result += self.freqvector[d][word]*(self.k + 1)*RSJ(word)/(self.k*(1-self.b + self.b*np.array(list(self.freqvector[d].values())).sum()/self.avg)+ self.freqvector[d][word])
      return result

    for d in self.freqvector.keys():
      ranking.append((d, f(q, d)))

    ranking.sort(key=lambda x: x[1], reverse=True)
    return ranking

class QLD:
  def __init__(self, freqvector, wordfreqs, mu):
    self.freqvector = freqvector
    self.wordfreqs = wordfreqs
    self.mu = mu

  def search(self, q):
    ranking = []
    def p(w):
      result = 0
      contador = 0
      for d, word_frec in self.freqvector.items():
        result += word_frec[w]
        contador += np.array(list(word_frec.values())).sum()
      return result/contador

    def f(q, d):
      result = 1
      for w in q:
        result *= (self.freqvector[d][w] + self.mu * p(w))/(np.array(list(self.freqvector[d].values())).sum() + self.mu)
      return result

    for d in self.freqvector.keys():
      ranking.append((d, f(q, d)))
    ranking.sort(key=lambda x: x[1], reverse=True)
    return ranking

Ahora procedemos a obtener el texto de las URLs del primer apartado. Utilizamos requests para obtener el texto y las procesamos mediante BeatifulSoup.

In [ ]:
# Indicación: el conjunto de URLs de esta práctica es qrels.docid.unique().
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from collections import Counter
import re

#Método que dijo el profe para conseguir las url
#urlopen(url).read()
urls = qrels.docid.unique()
valid_url = []
import requests
texts = []
for url in urls:
  try:
    response = requests.get(url)
    texts.append(BeautifulSoup(response.content, "lxml").text.lower())
    valid_url.append(url)
  except:
    print(f"La url: {url} no es accesible")

#texts = [BeautifulSoup(urlopen(url).read(), "lxml").text.lower() for url in urls]

stoplist = ["also", "could", "p", "pp", "th", "however", "one", "two", "many", "i", "de", "la", "me", "my", "myself", "the", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]



<ipython-input-21-03f654cf2cd5>:17: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  texts.append(BeautifulSoup(response.content, "lxml").text.lower())


La url: http://elsecretodeberlanga.rf.gd/2017/09/24/la-paradoja-del-gato-y-la-tostada/ no es accesible
La url: https://screenrant.com/twin-peaks-season-3-finale-ending-explained/ no es accesible
La url: https://collider.com/twin-peaks-guide/ no es accesible
La url: https://screenrant.com/twin-peaks-season-3-series-story-explained/ no es accesible
La url: https://collider.com/twin-peaks-season-3-finale-explained/ no es accesible
La url: https://aulaprimaria.com/blog/cuales-son-las-mejores-aperturas-con-blancas/ no es accesible
La url: https://thezugzwangblog.com/aperturas-recomendadas-segun-tu-nivel-de-juego/ no es accesible
La url: https://thezugzwangblog.com/la-apertura-catalana/ no es accesible
La url: https://thezugzwangblog.com/apertura-para-principiantes-con-blancas-contra-1-e5/ no es accesible


La mayoría de las páginas han sido procesadas salvo algunas de la query 2, 4, 7 y 9. A pesar de esto continuamos con el testeo puesto que, a pesar de ser relevantes, la mayoría de páginas sí se han procesado correctamente.

Pasamos ahora a procesar las queries y el texto de las páginas.

In [ ]:
queries = ["information theory in information retrieval",
"giant defy advanced pro 1 ultegra 2023 weight",
"que pasa cuando pego una tostada a un gato y lo tiro por la ventana",
"Autopsy training for stundets",
"vscode-clangd vs intellisense",
"What kind of bear is best",
"Twin Peaks Season 3 plot",
"como hacer el diagrama de Penrose de Reissner Nordstrom",
"cual es la mejor apertura de ajedrez con blancas"]

def tratamiento(q):
  aux = []
  for word in q.split():
    if word not in stoplist:
      aux.append(word)
  return aux

queries_tratado = [tratamiento(q) for q in queries]

In [ ]:
freqvector = {url:Counter([word for word in re.findall(r"[^\W\d_]+|\d+", text) if word not in stoplist]) for url, text in zip(valid_url, texts)}

# Guardamos el vocabulario (el conjunto de todas las palabras que apaercen en los documentos de la colección).
vocabulary = set()
for word in freqvector.values(): vocabulary.update(word)

# Document frequency de cada palabra del vocabulario: nº de documentos que contienen la palabra.
docfreqs = {word:len([url for url in freqvector if word in freqvector[url]]) for word in vocabulary}

# Frecuencia total para cada palabra del vocabulario: nº total de apariciones en la colección.
wordfreqs = {word:sum([freqvector[url][word] for url in freqvector if word in freqvector[url]]) for word in vocabulary}

Una vez hemos procesado todo el texto pasamos a generar los ránkings de para cada consulta y cada modelo.

In [ ]:
ranking_nuevo = pd.DataFrame(columns = ["system", "docid", "qid", "pos", "ranking", "rel"])

#Solo me quedaré con las 10 primeras posiciones

for q, qid in zip(queries_tratado, [0,1,2,3,4,5,7,8,9]):
  pos = 1
  for url, score in VSM(freqvector, docfreqs).search(q):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_nuevo.loc[len(ranking_nuevo)] = ["VSM",url , qid, pos, 11 - pos, rel]
      pos += 1
  pos = 1
  for url, score in BM25(freqvector, docfreqs, 0.5, 1).search(q):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_nuevo.loc[len(ranking_nuevo)] = ["BM25",url , qid, pos, 11 - pos, rel]
      pos += 1
  pos = 1
  for url, score in QLD(freqvector, docfreqs, 100).search(q):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_nuevo.loc[len(ranking_nuevo)] = ["QLD",url , qid, pos, 11 - pos, rel]
      pos += 1



Presentamos ahora los resultados:

In [ ]:
print("Precisión por query y buscador:")
print(prec(ranking_nuevo))

print("\nPrecisión media por buscador:")
print(prec_mean(ranking_nuevo))

print("\nnDCG por query y buscador:")
print(nDCG_(ranking_nuevo))

print("\nnDCG medio por buscador:")
print(nDCG_mean(ranking_nuevo))

print("\nRBP por query y buscador:")
print(RBP_(ranking_nuevo, 0.5))

print("\nRBP medio por buscador:")
print(RBP_mean(ranking_nuevo))

print("\nMeanRR por query y buscador:")
print(Mean_RR(ranking_nuevo))

print("\nMeanRR medio por buscador:")
print(Mean_RR_mean(ranking_nuevo))

print("\nMaxRR por query y buscador:")
print(Max_RR(ranking_nuevo))

print("\nMaxRR medio por buscador:")
print(Max_RR_mean(ranking_nuevo))

print("\nrecall por query y buscador:")
print(recall(ranking_nuevo))

print("\nrecall medio por buscador:")
print(recall_mean(ranking_nuevo))

print("\nmedia harmónica por query y buscador:")
print(harmonic(ranking_nuevo))

print("\nmedia harmónica media por buscador:")
print(harmonic_mean(ranking_nuevo))



Precisión por query y buscador:
system  qid
BM25    0      0.1
        1      0.6
        2      0.3
        3      0.2
        4      0.1
        5      0.5
        7      0.7
        8      0.1
        9      0.7
QLD     0      0.0
        1      0.7
        2      0.0
        3      0.0
        4      0.0
        5      0.0
        7      0.0
        8      0.0
        9      0.6
VSM     0      0.0
        1      0.8
        2      0.3
        3      0.2
        4      0.2
        5      0.6
        7      0.1
        8      0.0
        9      0.8
Name: rel, dtype: float64

Precisión media por buscador:
system
BM25    0.366667
QLD     0.144444
VSM     0.333333
Name: rel, dtype: float64

nDCG por query y buscador:
system  qid
BM25    0      0.315465
        1      0.974030
        2      0.807247
        3      0.674174
        4      0.289065
        5      0.773533
        7      0.661950
        8      0.500000
        9      0.701240
QLD     0           NaN
        1      0.77003

### Usando pyterrier

Una vez hemos procesado los datos usando nuestra propia implementación de las funciones de ránking, pasamos a utilizar los algoritmos de pyTerrier. Para ello comenzamos creando un índice a partir del texto procesado.

In [ ]:
terrier_docs = pd.DataFrame({'docno' : valid_url, 'url' : valid_url, 'text' : texts})
!rm -rf ./pd_index
pd_indexer = pt.DFIndexer("./pd_index")

In [ ]:
indexref = pd_indexer.index(terrier_docs["text"], terrier_docs["docno"])

Ahora generamos los modelos dado el indice:

In [ ]:
index = pt.IndexFactory.of(indexref)
vsm = pt.BatchRetrieve(index, wmodel='TF_IDF')
bm25 = pt.BatchRetrieve(index, wmodel='BM25')
qld = pt.BatchRetrieve(index, wmodel='DirichletLM')
pl2 = pt.BatchRetrieve(index, wmodel='PL2')
dph = pt.BatchRetrieve(index, wmodel='DPH')

In [ ]:
def printsearch(name, model, q):
  print('\n' + name)
  # Eliminamos todas las columnas del dataframe menos score y docno
  print((model).search(q)[['score', 'docno']].to_string(index=False))

def eval(names, models, queries, qrels, metrics, sort=[], baseline=None):
  # La clase Experiment ejecuta rankers sobre una batería de consultas, y calcula métricas.
  # El parámetro "baseline" hace que se añadan p-valores (y nº de consultas ganadas/perdidad) respecto a uno de los rankers.
  # Con el parámetro "sort" se ordena la tabla de métricas la métrica que se indique.
  print(pt.Experiment(models, queries, qrels, metrics, names, baseline=baseline).sort_values(str(sort), ascending=False).to_string(index=False))

Finalmente realizamos las predicciones y las evaluamos mediante las métricas implementadas en el apartado 1.

In [ ]:
ranking_pyterrier = pd.DataFrame(columns = ["system", "docid", "qid", "pos", "ranking", "rel"])

#Solo me quedaré con las 10 primeras posiciones

for q, qid in zip(queries_tratado, [0,1,2,3,4,5,7,8,9]):
  pos = 1
  q = " ".join(q)
  rank = (vsm%10).search(q)[['score', 'docno']]
  for url, score in zip(rank['docno'].values, rank['score'].values):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_pyterrier.loc[len(ranking_pyterrier)] = ["VSM (pyterrier)",url , qid, pos, 11 - pos, rel]
      pos += 1
  pos = 1
  rank = (bm25%10).search(q)[['score', 'docno']]
  for url, score in zip(rank['docno'].values, rank['score'].values):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_pyterrier.loc[len(ranking_pyterrier)] = ["BM25 (pyterrier)",url , qid, pos, 11 - pos, rel]
      pos += 1
  pos = 1
  rank = (qld%10).search(q)[['score', 'docno']]
  for url, score in zip(rank['docno'].values, rank['score'].values):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_pyterrier.loc[len(ranking_pyterrier)] = ["QLD (pyterrier)",url , qid, pos, 11 - pos, rel]
      pos += 1

  pos = 1
  rank = (pl2%10).search(q)[['score', 'docno']]
  for url, score in zip(rank['docno'].values, rank['score'].values):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_pyterrier.loc[len(ranking_pyterrier)] = ["PL2 (pyterrier)",url , qid, pos, 11 - pos, rel]
      pos += 1

  pos = 1
  rank = (dph%10).search(q)[['score', 'docno']]
  for url, score in zip(rank['docno'].values, rank['score'].values):
    if pos <= 10:
      rel = 0
      if ranking_rels[ranking_rels['docid'] == url]['qid'].values[0] == qid:
        rel = ranking_rels[ranking_rels["docid"] == url]["rel"].values[0]
      ranking_pyterrier.loc[len(ranking_pyterrier)] = ["DPH (pyterrier)",url , qid, pos, 11 - pos, rel]
      pos += 1

Eliminamos los rankings de la query 4 ya que no recuperan los suficientes documentos:

In [ ]:
ranking_pyterrier=ranking_pyterrier[ranking_pyterrier['qid']!=4]

In [ ]:
print("Precisión por query y buscador:")
print(prec(ranking_pyterrier))

print("\nPrecisión media por buscador:")
print(prec_mean(ranking_pyterrier))

print("\nnDCG por query y buscador:")
print(nDCG_(ranking_pyterrier))

print("\nnDCG medio por buscador:")
print(nDCG_mean(ranking_pyterrier))

print("\nRBP por query y buscador:")
print(RBP_(ranking_pyterrier))

print("\nRBP medio por buscador:")
print(RBP_mean(ranking_pyterrier))

print("\nMeanRR por query y buscador:")
print(Mean_RR(ranking_pyterrier))

print("\nMeanRR medio por buscador:")
print(Mean_RR_mean(ranking_pyterrier))

print("\nMaxRR por query y buscador:")
print(Max_RR(ranking_pyterrier))

print("\nMaxRR medio por buscador:")
print(Max_RR_mean(ranking_pyterrier))

print("\nrecall por query y buscador:")
print(recall(ranking_pyterrier))

print("\nrecall medio por buscador:")
print(recall_mean(ranking_pyterrier))

print("\nmedia harmónica por query y buscador:")
print(harmonic(ranking_pyterrier))

print("\nmedia harmónica media por buscador:")
print(harmonic_mean(ranking_pyterrier))


Precisión por query y buscador:
system            qid
BM25 (pyterrier)  0      0.0
                  1      0.7
                  2      0.3
                  3      0.3
                  5      0.5
                  7      0.9
                  8      0.2
                  9      0.8
DPH (pyterrier)   0      0.0
                  1      0.7
                  2      0.3
                  3      0.2
                  5      0.6
                  7      0.9
                  8      0.2
                  9      0.8
PL2 (pyterrier)   0      0.2
                  1      0.7
                  2      0.2
                  3      0.2
                  5      0.4
                  7      0.9
                  8      0.2
                  9      0.8
QLD (pyterrier)   0      0.2
                  1      0.7
                  2      0.2
                  3      0.2
                  5      0.6
                  7      0.9
                  8      0.2
                  9      0.7
VSM (pyterrier)   

Tras evaluar las métricas podemos determinar que en la mayoría de casos el modelo que mejor funciona es BM25 seguido de VSM (que lo supera en algunas) y de QLD.